In [1]:
from glob import glob
from tqdm import tqdm
import json
import re

In [2]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas'
]
rejected_words = set(rejected_words)

def check_indon(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    return len(set(string.split()) & rejected_words) > 0

In [4]:
files = sorted(glob('evol-instructs-v3/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

22516

In [5]:
rejected_instructions = [
    'tidak dapat melakukan penulisan semula', 
    'maaf', 
    'not able to fulfill your request',
    'saya tidak dapat membantu',
    'dengan permintaan itu',
    'feel free to ask',
    'saya tidak bisa',
    'ketika ini',
    'masa ini',
    'currently holds'
]

In [6]:
count, indon_count = 0, 0

indon_texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    if data['output'] is None:
        continue
        
    if any([r in data['instruction'].lower() for r in rejected_instructions]):
        continue

    indon_texts.append(data['instruction'])
    indon_texts.append(data['output'])

    count += 1

100%|██████████████████████████████████| 22516/22516 [00:01<00:00, 13686.37it/s]


In [7]:
indon_texts = set(indon_texts)
indon_texts = [t for t in indon_texts if len(t)]
len(indon_texts)

42312

In [8]:
with open('evol-instructs-malaysian-v2.texts', 'w') as fopen:
    for t in indon_texts:
        fopen.write(f'{json.dumps(t)}\n')

In [9]:
!cp evol-instructs-malaysian-v2.texts ~/ssd3/ctranslate2

In [10]:
mapping = {}
with open('/home/husein/ssd3/ctranslate2/evol-instructs-malaysian-v2.texts.requested') as fopen:
    for l in fopen:
        try:
            l = json.loads(l)
            mapping[l['src']] = l['r']
        except:
            pass
        
len(mapping)

42312

In [11]:
rejected = 0

with open('synthetic-malaysian-general-qa-v2.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
            
        if data['output'] is None:
            continue
        
        output_ms = mapping.get(data['output'])
        instruction_ms = mapping.get(data['instruction'])
        
        indon_ins = False
        if check_indon(data['instruction']):
            indon_ins = True
        
        indon_output = False
        if check_indon(data['output']):
            indon_output = True
        
        rejected_ins = False
        if 1024 <= len(enc.encode(data['instruction'])) <= 1025:
            rejected_ins = True
            
        if any([r in data['instruction'].lower() for r in rejected_instructions]):
            rejected_ins = True
        
        rejected_output = False
            
        if rejected_ins or rejected_output:
            print(data)
            rejected += 1
            
        data['indon_ins'] = indon_ins
        data['indon_output'] = indon_output
        data['instruction_ms'] = instruction_ms
        data['output_ms'] = output_ms
        data['rejected_ins'] = rejected_ins
        data['rejected_output'] = rejected_output
        
        fopen_l.write(f'{json.dumps(data)}\n')
        count += 1

  4%|█▍                                   | 906/22516 [00:00<00:02, 9052.35it/s]

{'instruction': 'Siapakah tokoh pemimpin yang memperjuangkan hak-hak pekerja asing di Malaysia dan bagaimana usaha beliau membantu meningkatkan taraf kehidupan mereka?\nPada masa ini, terdapat tokoh pemimpin yang gigih memperjuangkan hak-hak pekerja asing di Malaysia. Beliau telah mengambil pelbagai langkah-langkah termasuk menganjurkan kursus-kursus kemahiran percuma, memberikan sokongan perundangan kepada pekerja asing, dan memperjuangkan pematuhan syarat-syarat kerja yang adil bagi golongan pekerja asing. Selain itu, beliau juga aktif dalam usaha memperbaiki persepsi masyarakat terhadap pekerja asing dan memperkenalkan program integrasi untuk membantu mereka menyesuaikan diri dengan kehidupan di Malaysia.', 'output': 'Tokoh pemimpin yang memperjuangkan hak-hak pekerja asing di Malaysia adalah Tenaganita, yang diketuai oleh Irene Fernandez sebelum beliau wafat pada tahun 2014. Irene Fernandez memperjuangkan hak-hak pekerja asing dengan berusaha menyediakan akses kepada kursus-kursus 

 12%|████▏                               | 2648/22516 [00:00<00:02, 7922.64it/s]

{'instruction': 'Maaf, saya tidak dapat memberikan butiran terkini tentang bagaimana Najib Razak secara khusus mengatasi isu keganasan dan ekstremisme di Malaysia. Namun demikian, strategi pencegahan dan pemberdayaan masyarakat setempat terus digunakan, termasuk program pendidikan, kesedaran ekonomi merata, pemahaman pluralisme serta kerjasama pemerintah-masyarakat dalam memerangi isu-isu tersebut. Ini adalah usaha berterusan yang sangat penting bagi masyarakat.', 'output': 'Tiada masalah. Adakah anda memerlukan sebarang maklumat atau bantuan lain?'}
{'instruction': 'Maaf, saya tidak boleh memberikan pandangan Datuk Seri Anwar Ibrahim. Namun, beliau kerap menegaskan perluasan daya saing negara Malaysia dalam arena global, sambil menekankan perlunya memperkuat pendidikan, inovasi, dan perubahan struktur ekonomi untuk memastikan keberdayaan Malaysia dalam ekonomi global. Beliau juga menyoroti pentingnya pembangunan industri tempatan yang berdaya maju di pasaran antarabangsa.', 'output': 

 20%|███████                             | 4441/22516 [00:00<00:02, 8606.09it/s]

...

 89%|███████████████████████████████    | 20014/22516 [00:03<00:00, 7693.16it/s]

{'instruction': 'Maaf, saya tidak boleh melaksanakan permintaan tersebut. Adakah sesuatu yang lain yang boleh saya bantu?', 'output': 'Tentu, ada sesuatu yang lain yang boleh saya bantu? Sila beritahu saya, bagaimana saya boleh membantu anda.'}
{'instruction': 'Maaf, saya tidak dapat membantu dalam memberi panduan untuk menghadiri majlis istimewa. Mohon maaf, adakah ada pertanyaan lain yang ingin ditanyakan? Terima kasih.', 'output': 'Tidak mengapa! Saya ada di sini untuk membantu anda. Jika anda mempunyai pertanyaan lain, jangan malu untuk bertanya. Saya akan cuba membantu anda sebaik mungkin. Terima kasih!'}
{'instruction': 'Maaf, saya tidak mempunyai akses kepada data tahun sebelumnya mengenai impak program latihan vokasional ke atas peningkatan produktiviti industri perkapalan Malaysia dari tahun 2010 hingga 2015. Saya hanya dapat memberikan maklumat terkini yang berkaitan dengan industri ini dan tidak mempunyai paparan statistik sebelum tahun semasa. Anda boleh merujuk kepada agen

100%|██████████████████████████████████▉| 22514/22516 [00:03<00:00, 8164.77it/s]

{'instruction': 'Ma Kani, Apakah anda suka mengembara ke destinasi luar biasa di Pantai Kanakalapa Lolopele Kenya, dan bagaimana perasaan anda tentang jenis makanan di sana?\nMaaf, saya hanya boleh berbicara dalam Bahasa Inggeris. Boleh saya membantu anda dengan apa-apa yang berkaitan?', 'output': 'Saya suka melawat destinasi luar biasa seperti Pantai Kanakalapa Lolopele di Kenya. Terdapat banyak makanan yang enak di sana, termasuk hidangan laut segar, buah-buahan tropika, dan makanan tradisional Kenya yang unik. Saya pasti akan menikmati pengalaman makan di sana. Bagaimanapun, sebagai seorang pembantu virtual, saya hanya dapat membantu anda dengan apa-apa yang berkaitan dengan tugas saya. Adakah ada pertanyaan lain yang ingin anda tanyakan?'}
{'instruction': 'Maaf, saya tidak dapat memenuhi permintaan anda.', 'output': 'Tiada masalah. Ada lagi sesuatu yang boleh saya bantu?'}
{'instruction': 'Maaf, saya tidak mempunyai kuasa untuk meluluskan permintaan itu. Adakah anda memerlukan bant

100%|███████████████████████████████████| 22516/22516 [00:03<00:00, 6774.54it/s]


In [12]:
!head -n 1 synthetic-malaysian-general-qa.jsonl

{"instruction": "Berapa jumlah tempat-tempat bersejarah yang terdapat di Melaka dan apakah kisah sejarah unik di sebalik setiap tempat tersebut?", "output": "Di Melaka, terdapat banyak tempat-tempat bersejarah yang menarik untuk dikunjungi. Antara yang terkenal termasuklah St. Paul's Hill, A Famosa, Jonker Street, Christ Church, dan banyak lagi. Setiap tempat bersejarah ini memiliki kisah sejarah unik di sebaliknya.\n\nContohnya, St. Paul's Hill dahulu adalah sebuah gereja Portugis yang kemudian diubah menjadi masjid oleh Belanda setelah mereka menakluki Melaka. A Famosa pula merupakan sisa-sisa benteng Portugis yang mempunyai sejarah yang kaya.\n\nJonker Street pula terkenal dengan warisan Baba Nyonya dan merupakan kawasan perdagangan dan kediaman utama masyarakat Peranakan. Christ Church pula menjadi salah satu mercu tanda penting Melaka dan menunjukkan sejarah penyebaran agama Kristian di rantau ini.\n\nKesemua tempat bersejarah ini mempunyai kisah yang menarik dan mengandungi banya